<a href="https://colab.research.google.com/github/raginisharma1999/sentiment-monitor-iphone16pro/blob/main/sentiment_analysis_comments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =====================================================
# Quiet installs
# =====================================================
import sys, subprocess
subprocess.run(
    [sys.executable, "-m", "pip", "install", "-q",
     "supabase", "nltk", "gradio", "pandas", "matplotlib"],
    check=True
)

# =====================================================
# Imports
# =====================================================
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import gradio as gr
from supabase import create_client
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# =====================================================
# NLTK setup
# =====================================================
nltk.download("vader_lexicon", quiet=True)
vader = SentimentIntensityAnalyzer()

# =====================================================
# Constants (tunable, explicit)
# =====================================================
POSITIVE_THRESHOLD = 0.6
NEGATIVE_THRESHOLD = -0.4

# =====================================================
# Sentiment helpers
# =====================================================
def compute_sentiment(text):
    return vader.polarity_scores(text)["compound"]

def label_sentiment(score):
    if score >= 0.05:
        return "Positive"
    elif score <= -0.05:
        return "Negative"
    return "Neutral"

# =====================================================
# Data loading + processing
# =====================================================
def load_and_process(supabase_url, supabase_key):
    supabase = create_client(supabase_url, supabase_key)

    response = (
        supabase
        .table("comments")
        .select("comment, timestamp")
        .order("timestamp", desc=False)
        .execute()
    )

    df = pd.DataFrame(response.data)

    if df.empty:
        return None

    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df["compound"] = df["comment"].apply(compute_sentiment)
    df["sentiment"] = df["compound"].apply(label_sentiment)

    return df

# =====================================================
# Overview tab logic (READABLE trend)
# =====================================================
def run_dashboard(supabase_url, supabase_key):
    try:
        df = load_and_process(supabase_url, supabase_key)
        if df is None:
            return "No comments found.", None

        # Daily aggregation for readability
        df_daily = (
            df.set_index("timestamp")
              .resample("D")
              .mean(numeric_only=True)
        )

        plt.figure(figsize=(10, 4))
        plt.plot(df_daily.index, df_daily["compound"])
        plt.axhline(0, linestyle="--")
        plt.title("Average Daily Sentiment Trend")
        plt.xlabel("Date")
        plt.ylabel("Avg Compound Sentiment")
        plt.tight_layout()

        table_df = (
            df[["timestamp", "comment", "sentiment", "compound"]]
            .sort_values("timestamp", ascending=False)
            .reset_index(drop=True)
        )

        return table_df, plt

    except Exception as e:
        return f"Error: {str(e)}", None

# =====================================================
# Drill-down tab logic (MEANINGFUL extremes)
# =====================================================
def run_drilldown(supabase_url, supabase_key):
    try:
        df = load_and_process(supabase_url, supabase_key)
        if df is None:
            return "No data.", "No data."

        top_positive = (
            df[df["compound"] >= POSITIVE_THRESHOLD]
            .sort_values("compound", ascending=False)
            .head(2)[["timestamp", "comment", "compound"]]
            .reset_index(drop=True)
        )

        top_negative = (
            df[df["compound"] <= NEGATIVE_THRESHOLD]
            .sort_values("compound", ascending=True)
            .head(2)[["timestamp", "comment", "compound"]]
            .reset_index(drop=True)
        )

        return top_positive, top_negative

    except Exception as e:
        return f"Error: {str(e)}", f"Error: {str(e)}"

# =====================================================
# Gradio UI
# =====================================================
with gr.Blocks(title="Stakeholder Sentiment Monitoring Portal") as app:
    gr.Markdown("## 📊 Stakeholder Sentiment Monitoring Portal")

    with gr.Row():
        supabase_url = gr.Textbox(label="Supabase URL", type="password")
        supabase_key = gr.Textbox(label="Supabase Anon Key", type="password")

    with gr.Tabs():
        # ---------------- Overview ----------------
        with gr.Tab("Overview"):
            load_btn = gr.Button("Load Dashboard")

            table_output = gr.Dataframe(wrap=True)
            plot_output = gr.Plot()

            load_btn.click(
                fn=run_dashboard,
                inputs=[supabase_url, supabase_key],
                outputs=[table_output, plot_output]
            )

        # ---------------- Drill-down ----------------
        with gr.Tab("Drill-Down"):
            drill_btn = gr.Button("Load Top Reviews")

            positive_output = gr.Dataframe(label="Top 2 Most Positive Reviews", wrap=True)
            negative_output = gr.Dataframe(label="Top 2 Most Negative Reviews", wrap=True)

            drill_btn.click(
                fn=run_drilldown,
                inputs=[supabase_url, supabase_key],
                outputs=[positive_output, negative_output]
            )

app.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://184c7de54453f4aa1c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
